In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
pip install Flask

Note: you may need to restart the kernel to use updated packages.


In [8]:
# app.py
from flask import Flask, render_template

# Create a Flask app
app = Flask('Mental_Health_ASEAN2019')

# Define a route for the homepage ("/")
@app.route('/')
def home():
    """Serves the landing page."""
    # Define some data to potentially pass to the template
    page_title = "Welcome to Mental Health ASEAN Condition"
    headline = "Discover Mental Health Issues in 2019"
    description = "This is a simple landing page built with Python and Flask. Explore what we offer!"

# Render the index.html template, passing data to it
    return render_template(
        'index.html',
        title=page_title,
        headline=headline,
        description=description
    )

# Run the Flask development server when the script is executed directly
if 'Mental_Health_ASEAN2019' == '__main__':
    # debug=True allows for auto-reloading during development
    # host='0.0.0.0' makes it accessible on your network (optional)
    app.run(debug=True, host='0.0.0.0')

df.index[](https://www.kaggle.com/code/yasminauni/notebook6bed02e0ac/edit)

Map Connect

In [ ]:
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip, GeoJsonPopup

# --- Configuration ---
# 1. SET THE PATH TO YOUR QGIS LAYER FILE
#    (Shapefile, GeoJSON, GeoPackage, etc.)
qgis_layer_path = '"C:\Users\Lenovo\Documents\ArcGIS\Projects\MyProject5\Output\ASEAN_Export.shp.xml"'
# Example: qgis_layer_path = 'C:/Users/You/Documents/GIS/data/cities.shp'
# Example: qgis_layer_path = '/home/user/gis_data/countries.geojson'

# 2. SET THE OUTPUT HTML FILE NAME
output_html_path = 'Mental_Health_ASEAN2019_interactive_map.html'

# 3. CHOOSE COLUMNS FOR POPUPS AND TOOLTIPS
#    List the exact column names from your layer's attribute table
#    Popup appears on click, Tooltip appears on hover
popup_fields = ['name', 'population', 'country'] # ADJUST THESE to match your data
tooltip_fields = ['name'] # ADJUST THESE (often a key identifier)

# 4. (Optional) MAP STYLING
feature_color = '#3186cc' # Blue
feature_opacity = 0.7
highlight_color = 'yellow' # Color when hovering

# 5. (Optional) MAP STARTING VIEW
#    If None, it will try to center on the data. Otherwise, set manually.
#    Format: [latitude, longitude]
map_start_location = None # Example: [40.7128, -74.0060] # New York City
map_start_zoom = None    # Example: 10 (Higher number = more zoomed in)

# --- Load Data ---
print(f"Loading data from: {qgis_layer_path}")
try:
    gdf = gpd.read_file(qgis_layer_path)
    print(f"Loaded {len(gdf)} features.")
    print(f"Original CRS: {gdf.crs}")
except Exception as e:
    print(f"ERROR: Could not load the file '{qgis_layer_path}'.")
    print(f"Details: {e}")
    exit() # Stop the script if the file can't be loaded

# --- Data Preparation ---
# Ensure data is in WGS84 (EPSG:4326) - standard for web maps
if gdf.crs is None:
    print("WARNING: Layer has no CRS defined. Assuming EPSG:4326 (WGS84).")
    # If you *know* it's WGS84 but lacks the definition:
    # gdf = gdf.set_crs(epsg=4326, allow_override=True)
    # If you know it's something else, define it first, then reproject:
    # gdf = gdf.set_crs(epsg=YOUR_EPSG_CODE).to_crs(epsg=4326)
elif gdf.crs.to_epsg() != 4326:
    print(f"Reprojecting data from {gdf.crs.name} (EPSG:{gdf.crs.to_epsg()}) to WGS84 (EPSG:4326)...")
    try:
        gdf = gdf.to_crs(epsg=4326)
        print("Reprojection successful.")
    except Exception as e:
        print(f"ERROR: Failed to reproject data. {e}")
        # Decide if you want to continue with potentially misplaced data or exit
        # exit()

# Check if specified popup/tooltip fields actually exist in the data
available_columns = list(gdf.columns)
popup_fields = [f for f in popup_fields if f in available_columns]
tooltip_fields = [f for f in tooltip_fields if f in available_columns]
if not popup_fields:
    print("WARNING: None of the specified popup_fields exist in the layer.")
if not tooltip_fields:
    print("WARNING: None of the specified tooltip_fields exist in the layer.")


# --- Create Map ---
# Determine map center and zoom
if map_start_location is None:
    # Calculate the center of the layer's bounding box
    try:
        bounds = gdf.total_bounds # [minx, miny, maxx, maxy]
        center_lat = (bounds[1] + bounds[3]) / 2
        center_lon = (bounds[0] + bounds[2]) / 2
        map_location = [center_lat, center_lon]
        # Basic zoom guess (optional, Folium often does okay automatically)
        if map_start_zoom is None:
             map_zoom = 6 # Default zoom if not specified
        else:
             map_zoom = map_start_zoom

    except Exception as e:
         print(f"Warning: Could not calculate bounds/center automatically ({e}). Falling back to default view.")
         map_location = [20, 0] # Default fallback location
         map_zoom = 2         # Default fallback zoom
else:
    map_location = map_start_location
    map_zoom = map_start_zoom if map_start_zoom is not None else 10 # Default zoom if location is set but zoom isn't


print(f"Creating map centered at {map_location} with zoom level {map_zoom}")
m = folium.Map(location=map_location, zoom_start=map_zoom, tiles='OpenStreetMap')

# --- Add Data Layer to Map ---
print("Adding data layer to the map...")

# Style function for features
style_function = lambda x: {
    'fillColor': feature_color,
    'color': 'black',       # Border color
    'weight': 1,            # Border width
    'fillOpacity': feature_opacity,
}

# Highlight function for when mouse hovers over feature
highlight_function = lambda x: {
    'fillColor': highlight_color,
    'color': 'black',
    'weight': 3,
    'fillOpacity': 0.8,
}

# Create the GeoJson layer with tooltips and popups
geojson_layer = folium.GeoJson(
    gdf, # Your GeoDataFrame
    name='My QGIS Data', # Layer name for LayerControl
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=GeoJsonTooltip(
        fields=tooltip_fields,
        aliases=[f"{field.replace('_', ' ').title()}:" for field in tooltip_fields], # Nicer labels
        sticky=True # Tooltip follows mouse
    ) if tooltip_fields else None, # Only add tooltip if fields are specified
    popup=GeoJsonPopup(
        fields=popup_fields,
        aliases=[f"{field.replace('_', ' ').title()}:" for field in popup_fields], # Nicer labels
        max_width=300,
    ) if popup_fields else None, # Only add popup if fields are specified
    # embed=False # For very large datasets, setting embed=False can sometimes improve performance
                  # by linking to an external GeoJSON file instead of embedding it in the HTML.
                  # Folium handles creating this extra file automatically when you save.
)

geojson_layer.add_to(m)

# --- Add Layer Control ---
# Allows users to toggle layers on/off (useful if you add more layers)
folium.LayerControl().add_to(m)

# --- Save Map ---
print(f"Saving interactive map to: {output_html_path}")
try:
    m.save(output_html_path)
    print("Map saved successfully!")
except Exception as e:
    print(f"ERROR: Could not save the map. {e}")

print(f"\nProcess finished. Open '{output_html_path}' in your web browser.")